# CUNEF MUCD 2022/2023
## Machine Learning
## Análisis de Fraude

# Autores:
-Gozde Yazganoglu
-Irma Sanchez

In [73]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import ColumnTransformer

warnings.filterwarnings('ignore')

from aux_func import evaluate_model
from aux_func import model_analysis, cargar_modelo
import lightgbm as lgb
import pickle

In [74]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from aux_func import model_analysis, cargar_modelo

In [75]:
preprocessor = cargar_modelo('../models/preprocessor.pickle')

In [76]:
xtrain = pd.read_parquet("../data/processed/xtrain.parquet")
ytrain = pd.read_parquet("../data/processed/ytrain.parquet")
xtest = pd.read_parquet("../data/processed/xtest.parquet")
ytest = pd.read_parquet("../data/processed/ytest.parquet")

## OPTIMIZACIÓN RANDOM FOREST

In [94]:

rf = Pipeline(steps=[
    ('preprocesador', preprocessor),
    ('clasificador', RandomForestClassifier())])

### Parameter Tunning Optimization
param_grid = { 
    'clasificador__n_estimators': [200, 250, 500],
    'clasificador__max_features': ['sqrt', 'log2'],
    'clasificador__max_depth' : [4, 6, 8, None],
    'clasificador__criterion' :['gini'],
    'clasificador__class_weight':['balanced']}


CV = GridSearchCV(rf, param_grid, cv=3, n_jobs=8, scoring='roc_auc', verbose=1)

In [22]:
evaluate_model(ytest, ypred, ypred_proba)

ROC-AUC score of the model: 0.9950900106098783
Accuracy of the model: 0.9862956870037909

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    209487
           1       0.07      0.94      0.13       228

    accuracy                           0.99    209715
   macro avg       0.53      0.96      0.56    209715
weighted avg       1.00      0.99      0.99    209715


Confusion matrix: 
[[206627   2860]
 [    14    214]]



In [ ]:
CV.fit(xtrain, ytrain)
print(CV.best_params_)
print(CV.best_score_)


Fitting 3 folds for each of 24 candidates, totalling 72 fits


In [ ]:

with open('../models/RandomForest_optimizado.pickle', 'wb') as f:
    pickle.dump(CV, f)


In [ ]:
with open('../models/RandomForest_optimizado.pickle', 'rb') as f:
    CV = pickle.load(f)

In [ ]:
print(CV.best_params_)
print(CV.best_score_)
ypred = CV.predict(xtest)
ypred_proba = CV.predict_proba(xtest)

In [ ]:
evaluate_model(ytest, ypred, ypred_proba)

In [ ]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)
